In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import datetime


In [2]:
# Load the data from gas.csv
df = pd.read_csv('gas1.csv')

In [3]:
df

,Location,Month,Prices
0,"Charlottetown and Summerside, Prince Edward Is...",Jan-90,50.3
1,"Charlottetown and Summerside, Prince Edward Is...",Feb-90,50.5
2,"Charlottetown and Summerside, Prince Edward Is...",Mar-90,50.8
3,"Charlottetown and Summerside, Prince Edward Is...",Apr-90,51.1
4,"Charlottetown and Summerside, Prince Edward Is...",May-90,51.7
...,...,...,...
6947,"Yellowknife, Northwest Territories",Aug-23,175.7
6948,"Yellowknife, Northwest Territories",Sep-23,180.3
6949,"Yellowknife, Northwest Territories",Oct-23,183.1
6950,"Yellowknife, Northwest Territories",Nov-23,187.3


In [4]:
# Convert 'Month' column to datetime format
df['Month'] = pd.to_datetime(df['Month'], format='%b-%y')


In [5]:
# Extract year and month as separate numerical features
df['Year'] = df['Month'].dt.year
df['Month_Num'] = df['Month'].dt.month

In [6]:
# Clean the data by removing rows with non-numeric values in 'prices' column
df = df[pd.to_numeric(df['Prices'], errors='coerce').notnull()]


In [7]:
# Convert 'prices' column to float
df['Prices'] = df['Prices'].astype(float)


C:\Users\User\AppData\Local\Temp\ipykernel_20560\497285554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Prices'] = df['Prices'].astype(float)


In [8]:
# Group the data by 'Location' for predictions
groups = df.groupby('Location')


In [9]:
# Initialize an empty DataFrame to store the predictions
predicted_df = pd.DataFrame()


In [10]:
# Iterate through each group and make predictions
for name, group in groups:
    # Splitting the data into features (X) and target variable (y)
    X = group[['Year', 'Month_Num']]
    y = group['Prices']

    # Training a linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Create a DataFrame for future dates for this location
    future_dates = pd.date_range(start=group['Month'].max(), periods=60, freq='M') + datetime.timedelta(days=30)
    future_df = pd.DataFrame({'Month': future_dates})
    future_df['Year'] = future_df['Month'].dt.year
    future_df['Month_Num'] = future_df['Month'].dt.month

    # Make predictions for this location
    future_predictions = model.predict(future_df[['Year', 'Month_Num']])

    # Add predictions to the predicted DataFrame
    future_df['Predicted Prices'] = future_predictions
    future_df['Location'] = name

    # Append the predictions for this location to the overall predicted DataFrame
    predicted_df = pd.concat([predicted_df, future_df])


In [11]:
# Round predicted prices to 1 decimal place
predicted_df['Predicted Prices'] = predicted_df['Predicted Prices'].round(1)

# Save predicted DataFrame to a CSV file
predicted_df.to_csv('Predicted_Diesel_Prices.csv', index=False)

In [12]:
# Reset the index of the predicted DataFrame
predicted_df.reset_index(drop=True, inplace=True)
predicted_df

,Month,Year,Month_Num,Predicted Prices,Location
0,2024-01-30,2024,1,140.0,"Calgary, Alberta"
1,2024-03-01,2024,3,140.9,"Calgary, Alberta"
2,2024-03-30,2024,3,140.9,"Calgary, Alberta"
3,2024-04-30,2024,4,141.4,"Calgary, Alberta"
4,2024-05-30,2024,5,141.8,"Calgary, Alberta"
...,...,...,...,...,...
1015,2028-08-30,2028,8,174.5,"Yellowknife, Northwest Territories"
1016,2028-09-30,2028,9,175.0,"Yellowknife, Northwest Territories"
1017,2028-10-30,2028,10,175.4,"Yellowknife, Northwest Territories"
1018,2028-11-30,2028,11,175.9,"Yellowknife, Northwest Territories"


In [13]:
import hvplot.pandas
# Plotting using hvplot
plot = predicted_df.hvplot.line(x='Month', y='Predicted Prices', by='Location', xlabel='Month', ylabel='Price (Cents per Liter)', title='Diesel Prices Prediction', hover_cols=['Location'])

# Show the plot
plot

:NdOverlay   [Location]
   :Curve   [Month]   (Predicted Prices,Location)